In [75]:
# compass

from datetime import datetime
# Import Required Library
from tkinter import *
from tkcalendar import Calendar

date = datetime.now()
date_year = int(date.strftime("%Y"))
date_month = int(date.strftime("%m"))
date_day = int(date.strftime("%d"))


# Create Object
root = Tk()

# Set geometry
root.geometry("400x400")

# Add Calendar
cal = Calendar(root, selectmode = 'day',
			year = date_year, month = date_month,
			day = date_day)

cal.pack(pady = 20)

def grad_date():
	date.config(text = "Selected Date is: " + cal.get_date())

# Add Button and Label
Button(root, text = "Get Date",
	command = grad_date).pack(pady = 20)

date = Label(root, text = "")
date.pack(pady = 20)

# Execute Tkinter
root.mainloop()


In [122]:
from geopy.geocoders import Nominatim

# Initialize Nominatim geocoder
geolocator = Nominatim(user_agent="my_app")

# Get your current location
location = geolocator.geocode("Tehran")

# Print the latitude and longitude of your current location
if location:
    print("Latitude:", location.latitude)
    print("Longitude:", location.longitude)
else:
    print("Location not found")

Latitude: 35.6892523
Longitude: 51.3896004


In [107]:
from geopy.distance import geodesic
import geopy

work = geopy.Point(41.4107628, 2.1745004)
home = geopy.Point(41.4126728, 2.1704725)

print(geodesic(home, work).kilometers)  # Output distance in meters

0.3980110171828715


In [18]:
from geopy.distance import geodesic
import math

# Your location
your_location = (41.4107628, 2.1745004)

# Target point
target_point = (41.4126728, 2.1704725)

# Calculate the bearing angle
distance = geodesic(your_location, target_point).meters
delta_y = target_point[1] - your_location[1]
delta_x = target_point[0] - your_location[0]

bearing = math.degrees(math.atan2(delta_x, delta_y))

if bearing < 0:
    bearing += 360

print(f"Bearing angle from your location to the target point: {bearing} degrees")

Bearing angle from your location to the target point: 154.6300425458961 degrees


In [21]:
def classify_direction(angle):
    directions = ["North", "Northeast", "East", "Southeast", 
                  "South", "Southwest", "West", "Northwest"]

    index = round(angle / (360. / len(directions))) % len(directions)
    
    return directions[index]

# Example bearing angle (0 to 360 degrees)
bearing_angle = 45 

direction = classify_direction(bearing_angle)

print(f"The direction based on the bearing angle {bearing_angle} degrees is: {direction}")

The direction based on the bearing angle 45 degrees is: Northeast


In [144]:
def get_points(d_city):
    geolocator = Nominatim(user_agent="my_app")
    location = geolocator.geocode(d_city)
    if location:
        return location.latitude, location.longitude

get_points("my_location")

(-1.9893346, 30.0861036)

In [145]:
def find_angles(target_city):
    your_location = get_points("my_location")
    your_location = (your_location[0], your_location[1])
    target_location = get_points(target_city)
    target_location = (target_location[0], target_location[1])
    print(your_location)
    
    if your_location is None or target_location is None:
        print("Error: Unable to find location for one or both cities.")
        return None
    
    distance = geodesic((your_location), (target_location)).kilometers
    delta_y = target_location[1] - your_location[1]
    delta_x = target_location[0] - your_location[0]

    bearing = math.degrees(math.atan2(delta_x, delta_y))

    if bearing < 0:
        bearing += 360
    
    return bearing, distance

print(find_angles("Isfahan"))

(-1.9893346, 30.0861036)
(58.094197313980395, 4458.500104569072)


In [24]:
from datetime import datetime
from tkinter import *
from tkcalendar import Calendar
import tkinter.ttk as ttk
import math
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import tkinter as tk
from PIL import ImageTk, Image
import requests

def get_points(d_city):
    geolocator = Nominatim(user_agent="my_app")
    location = geolocator.geocode(d_city)
    if location:
        return location.latitude, location.longitude


def find_angles(target_city):
    response = requests.get('http://ip-api.com/json/')
    location = response.json()
    my_current = (location['lat'], location['lon'])
    target_location = get_points(target_city)
    target_location = (target_location[0], target_location[1])

    if my_current is None or target_location is None:
        print("Error: Unable to find location for one or both cities.")
        return None
    
    distance = geodesic((my_current), (target_location)).kilometers
    delta_y = target_location[1] - my_current[1]
    delta_x = target_location[0] - my_current[0]

    bearing = math.degrees(math.atan2(delta_x, delta_y))

    if bearing < 0:
        bearing += 360
    
    return bearing, distance
    

def classify_direction(angle):
    directions = ["North", "Northeast", "East", "Southeast", 
                  "South", "Southwest", "West", "Northwest"]

    index = round(angle / (360. / len(directions))) % len(directions)
    
    return directions[index]


def calculate_destination_direction(destination):
    global combobox_destination_city
    d_city = get_points(destination)
    
    angle_c_d = find_angles(destination)[0]
    angle_c_d_1=find_angles(destination)[1]

    classify_dirs = classify_direction(angle_c_d)

    r = tk.Toplevel()
    r.geometry("928x576")
    r.title("result page")
    canvas= Canvas(r, width= 928, height= 576)
    canvas.pack()
    #Load an image in the script
    img= ImageTk.PhotoImage(Image.open("island.jpg"))
    #Add image to the Canvas Items
    canvas.create_image(0,0,anchor=NW,image=img)

    label_result_1 = tk.Label(r, text=f"City : {combobox_destination_city.get()} ", bg="#27187E", fg="#AEB8FE", font=("Arial", 25))
    label_result_1.place(x=30, y=50)
    label_result_2 = tk.Label(r, text=f"Direction : {classify_dirs} ", bg="#27187E", fg="#AEB8FE", font=("Arial", 25))
    label_result_2.place(x=30, y=120)
    label_result_3 = tk.Label(r, text=f"Distance : {angle_c_d_1//1000} kilometers ", bg="#27187E", fg="#AEB8FE", font=("Arial", 25))
    label_result_3.place(x=30, y=190)
    r.mainloop()

    print(classify_dirs)
    print(angle_c_d_1)
    return classify_dirs, angle_c_d_1

    

date = datetime.now()
date_year = int(date.strftime("%Y"))
date_month = int(date.strftime("%m"))
date_day = int(date.strftime("%d"))

root = Tk()
root.geometry("600x600")
root.configure(background="#AEB8FE")

border_combo = Label(root, bg="#27187E", font=("Arial", 16), fg="#F1F2F6")
border_combo.place(x= 241, y=410, height=55, width=265)
combobox_destination_city = ttk.Combobox(root, font=("Arial", 15), values=["Tehran", "Mashhad", "Isfahan", "Tabriz", "Shiraz", "Ahvaz", "Qom", "Kahriz", "Kermanshah",
               "Urmia", "Zahedan", "Qazvin", "Kerman", "Ardabil", "Rasht", "Sari", "Arak", "Bandar Abbas",
               "Yasuj", "Abadan", "Bojnourd", "Dezful", "Birjand", "Ilam", "Zanjan", "Malayer", "Saveh", 
               "Shahrud", "Mahabad", "Bandar-e Anzali", "Yazd", "Babol", "Borujerd", "Semnan", "Gonbad-e Kavus",
               "Varamin", "Gorgan", "Quchan", "Hamadan", "Maragheh", "Shahrekord", "Ghaemshahr", "Shahroud", 
               "Masjed Soleyman", "Shushtar", "Ramsar", "Behbahan", "Tonekabon", "Borazjan", "Bastam"], height=6)
combobox_destination_city.place(x=250, y=420)

t_city = Label(root, text="Target city : ", bg="#27187E", font=("Arial", 16), fg="#F1F2F6")
t_city.place(x= 85, y=420)

accept_button = Button(root, text="Accept", bg="#27187E", font=("Arial", 15), fg="#F1F2F6", activebackground="black", activeforeground="white",
                        command=lambda:calculate_destination_direction(combobox_destination_city.get()))
accept_button.place(x=250, y=495, width=100, height=35)


date = Label(root, text="Calendar", bg="#27187E", font=("Arial", 25), fg="#F1F2F6")
date.place(x= 240, y=7)

# Create a custom style for the calendar
style = ttk.Style()
style.theme_use('winnative')  # Change the theme if needed

# Configure the colors for different parts of the calendar
style.map("TButton",
          background=[('active', 'black')],
          foreground=[('active', 'black')])

cal = Calendar(root, selectmode='day',
               year=date_year, month=date_month, day=date_day, background="#27187E", disabledbackground="black", bordercolor="black", 
               headersbackground="#FF8600", normalbackground="#758BFD", foreground='white', 
               normalforeground='#27187E', headersforeground='#27187E', font="Arial 17", locale='en_US')

cal.place(x=85, y=90)

def grab_date():
    date.config(text="Selected Date is: " + cal.get_date())

# Button(root, text="Get Date", command=grab_date).pack(pady=20)

# date = Label(root, text="")
# date.pack(pady=20)

root.mainloop()

In [ ]:
# these are thems to style your calendar. => style.theme_use("your_style")

# 'classic': Provides a classic look and feel for widgets.
# 'alt': Alternate appearance for the widgets.
# 'default': Default theme based on the platform’s native theme.
# 'vista': Theme with a Vista-like appearance.
# 'xpnative': Mimics the appearance of Windows XP widgets.
# 'winnative': Emulates the appearance of Windows widgets.
# 'clam': Provides a simple and clean look, somewhat similar to Python’s default IDLE interface.
# 'black': A theme with a black background and white accents.

# You can use these theme names with the ttk.Style.theme_use() method to switch the appearance of Tkinter widgets based on your 
# preferences. Feel free to experiment with these themes to find the one that best suits your application’s design.


In [1]:
import tkinter as tk
from tkinter import *
from tkcalendar import Calendar
import tkinter.ttk as ttk
import math
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import tkinter as tk
from PIL import ImageTk, Image
from tkinter import Canvas
r = tk.Tk()
r.geometry("928x576")
canvas= Canvas(r, width= 928, height= 576)
canvas.pack()

#Load an image in the script
img= ImageTk.PhotoImage(Image.open("island.jpg"))

#Add image to the Canvas Items
canvas.create_image(0,00,anchor=NW,image=img)
r.mainloop()

In [22]:
import geopy.distance

response = requests.get('http://ip-api.com/json/')
location = response.json()
coords_1 = (location['lat'], location['lon'])
coords_2 = get_points("Mashhad")

print(coords_1)

print(geopy.distance.geodesic(coords_1, coords_2).km)

(39.0438, -77.4874)
10576.951028554191


In [18]:
import mpu
dist = mpu.haversine_distance(coords_1, coords_2)
print(dist)

10553.535190569952


In [19]:
from haversine import Unit
import haversine as hs
#To calculate distance in meters 
hs.haversine(coords_1,coords_2,unit=Unit.KILOMETERS)

10553.549767733613

In [27]:
from tkinter import *
from geopy.geocoders import Nominatim

# Create an instance of tkinter frame
win = Tk()

# Define geometry of the window
win.geometry("700x350")

# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")

# Latitude & Longitude input
coordinates = "39.0438 , -77.4874"

location = geolocator.reverse(coordinates)

address = location.raw['address']

# Traverse the data
city = address.get('city', '')
state = address.get('state', '')
country = address.get('country', '')

# Create a Label widget
label1=Label(text="Given Latitude and Longitude: " + coordinates, font=("Calibri", 24, "bold"))
label1.pack(pady=20)

label2=Label(text="The city is: " + city, font=("Calibri", 24, "bold"))
label2.pack(pady=20)

label3=Label(text="The state is: " + state, font=("Calibri", 24, "bold"))
label3.pack(pady=20)

label4=Label(text="The country is: " + country, font=("Calibri", 24, "bold"))
label4.pack(pady=20)

win.mainloop()

In [ ]:
# understand my ip
from urllib.request import urlopen
import json

url = "http://ipinfo.io/json"
response = urlopen(url)
data = json.load(response)
print((data['loc'])) # my current lat and lon

In [52]:
t = data['loc'].split(",")
t[0], t[1] = float(t[0]), float(t[1])
t = tuple(t)
t

(39.0853, -77.6452)

In [ ]:
import geopy.distance

coords_1 = t
coords_2 = get_points("Mashhad")

print(coords_1)

print(geopy.distance.geodesic(coords_1, coords_2).km)


In [10]:
import requests

# Get current location
response = requests.get('http://ip-api.com/json/')
location = response.json()
my_current = (location['lat'], location['lon'])
print(my_current)
print("Latitude:", location['lat'])
print("Longitude:", location['lon'])

(34.6432, 50.9084)
Latitude: 34.6432
Longitude: 50.9084


In [38]:
from geopy.geocoders import ArcGIS

# Create a geolocator object
geolocator = ArcGIS()

# Get current location using the geolocator
location = get_points("Qom")
coords_2 = get_points("Isfahan")
print(coords_2)
print(location)



print(geopy.distance.geodesic(location, coords_2).km)


# Print out the location details
# print("City:", location.address)

# print("Latitude:", location.latitude)
# print("Longitude:", location.longitude)

(32.6707877, 51.6650002)
(34.6422939, 50.8801184)
230.46975126488186


In [39]:
from haversine import Unit
import haversine as hs
#To calculate distance in meters 
hs.haversine(location,coords_2,unit=Unit.KILOMETERS)

230.94225117909858

In [47]:
import geocoder

# Get location details based on IP address
ip_info = geocoder.ip('me')

# Print out the location details
print("City:", ip_info.city)
print("Region:", ip_info.region)
print("Country:", ip_info.country)
print("Latitude:", ip_info.lat)
print("Longitude:", ip_info.lng)

Status code 403 from http://ipinfo.io/json: ERROR - 403 Client Error: Forbidden for url: http://ipinfo.io/json


City: None
Region: None
Country: None
Latitude: None
Longitude: None
